In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Constants
DS = "../data/HR Employee Attrition.csv"
RANDOM_SEED = 42

# Read data set
df = pd.read_csv(filepath_or_buffer=DS)

#### Data Cleaning
- Indentify which columns have non numerical data
- Convert NaN data to numerical

In [ ]:
# Identifying NaN Columns
NaN_columns = df.select_dtypes(exclude='number').columns.tolist()
for col in NaN_columns:
    print(col)
    print(df[col].unique())

In [ ]:
# Numerizing NaN columns
df["Attrition"].replace({"Yes":1, "No":0}, inplace=True )
df["OverTime"].replace( {"Yes":1, "No":0}, inplace=True )
df["Over18"].replace( {"Y":1}, inplace=True)
df["BusinessTravel"].replace( {"Travel_Rarely":0.5, "Travel_Frequently":1, "Non-Travel":0}, inplace=True )
df["Gender"].replace( {"Female":1, "Male":0}, inplace=True )
df["MaritalStatus"].replace( {"Single":0, "Married":1, "Divorced":2}, inplace=True )
# Note: Columns [Department], [JobRole], [EducationField] wont be an input.
df.drop(columns=["Department", "JobRole", "EducationField"], inplace=True)

#### Setting a Random Forest Regressor
Uill be used to get importance levels of all the features of the dataset

In [ ]:
def get_feature_importance(random_seed:int, data_set:pd.DataFrame, target_feature:str):
    # Separate traget column
    X = data_set.drop(target_feature, axis=1)
    y = data_set[target_feature]

    # Set model
    model = RandomForestRegressor(random_state=random_seed)
    model.fit(X, y)

    # Get feature importance
    feature_importances = model.feature_importances_
    feature_names = X.columns
    importance_dict = dict(zip(feature_names, feature_importances))

    return np.array(sorted(importance_dict.items(), key=lambda x: x[1], reverse=True))

#### Experiment 1
Find out which columns/employee attributes, contribute the most to employee attrition?

In [ ]:
# Get feature importance scores
ft_importance = get_feature_importance(data_set=df, random_seed=RANDOM_SEED, target_feature="Attrition")
# Higher score means that columns affects the target column the most

In [ ]:
# Graphing the 10 most influential columns
ft = ft_importance[:,1].astype("float") # str to float
ft = (ft / np.sum(ft))[:10]             # Normalize scores
plt.bar(ft_importance[:10,0], ft)
plt.xticks(rotation="vertical")
plt.xlabel('Feature')
plt.ylabel('Normalized Importance Score')
plt.title('Normalized Feature Importance Scores')
# Save Grpah
graph_path = "../graph/Feature-Importance-Graph.png"
plt.savefig(graph_path)
# Show graph
plt.figure(figsize=(10,10))
plt.show()


#### Experiment 2
Given the attributes of an employee, predict attrition.

In [ ]:
# Extract features (X) and target variable (y)
X = df.drop(["Attrition"], axis=1)
y = df['Attrition']

# one-hot encoding
X = pd.get_dummies(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

# Initialize the RandomForestClassifier
model = RandomForestClassifier(random_state=RANDOM_SEED)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)


In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

In [49]:
def run_attrition_simulation(model:RandomForestClassifier):
    # This is for future works.
    # Not fully implemented yet.
    attributes_required = ft_importance[:,0][:10]
    emp_info = pd.DataFrame()
    for idx, att in enumerate(attributes_required):
        user_input = input(f"Enter {att}: ")
        emp_info[att] = [float(user_input)]

    print(emp_info)
    prediction = model.predict(emp_info)
    print(f"Attrition: {prediction}")
    return 


### Experiment 3
Which departments have the highest attrition levels?

In [51]:
attrition_by_department = df.groupby('Department')['Attrition'].value_counts(normalize=True).unstack()
department_highest_attrition = attrition_by_department[1].sort_values(ascending=False)

# Display the department with the highest attrition
print("Department with the Highest Attrition:")
print(department_highest_attrition.head(1))

KeyError: 'Department'